<a href="https://colab.research.google.com/github/AnkeHooliganke/bases/blob/main/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import json
from pyspark.sql.functions import col

In [4]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("Pyspark Execcises") \
                    .getOrCreate()

sc = spark.sparkContext

In [5]:
data_path = "/content/german.data"

In [7]:
# if you don't have wget (on a win machine, for example) just download the data manually
# and put into $data_path
def download_data_if_needed(data_path):
    DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
    if not os.path.isfile(data_path):
        import wget
        wget.download(DATASET_URL, "data/")
    else:
        print('data is already in place')

In [8]:
download_data_if_needed(data_path)

data is already in place


In [6]:
def show(df):
    return df.limit(7).toPandas().head() # читерим в целях демонстрации

In [16]:
sch = [('existingchecking', StringType()),  #1
       ('duration',IntegerType()), # 2
       ('credithistory',StringType()), # 3
       ('purpose',StringType()), # 4
       ('creditamount',IntegerType()), # 5
       ('savings',StringType()), # 6
       ('employmentsince',StringType()),  # 7
       ('installmentrate',IntegerType()), # 8
       ('statussex',StringType()),  # 9
       ('otherdebtors',StringType()),  # 10
       ('residencesince',IntegerType()), # 11
       ('property',StringType()),  # 12
       ('age',IntegerType()), # 13
       ('otherinstallmentplans',StringType()),  # 14
       ('housing',StringType()),  # 15
       ('existingcredits',IntegerType()), # 16
       ('job',StringType()),  # 17
       ('peopleliable',IntegerType()), # 18
       ('telephone',StringType()),  # 19
       ('foreignworker',StringType()),  # 20
       ('classification', IntegerType())]

# wrapper 
schema = StructType([StructField(name, column_type, True) for name, column_type in sch])


# TASK 1: read data

df = spark.read.option("delimiter", ' ').csv(data_path, schema=schema)

show(df)

,existingchecking,duration,credithistory,purpose,creditamount,savings,employmentsince,installmentrate,statussex,otherdebtors,residencesince,property,age,otherinstallmentplans,housing,existingcredits,job,peopleliable,telephone,foreignworker,classification
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2


In [17]:
with open("/content/german_mapping.json", "r") as mapping_file:
    mapping = json.load(mapping_file)

In [18]:
# TASK 2: remap values

df = df.replace(to_replace=mapping)

In [ ]:
# df.show() -- плохо печатает не в терминале

In [19]:
show(df)

,existingchecking,duration,credithistory,purpose,creditamount,savings,employmentsince,installmentrate,statussex,otherdebtors,residencesince,property,age,otherinstallmentplans,housing,existingcredits,job,peopleliable,telephone,foreignworker,classification
0,... < 0 DM,6,critical account/other credits existing (not a...,radio/television,1169,unknown/ no savings account,>= 7 years,4,male - single,none,4,real estate,67,none,own,2,skilled employee/official,1,"yes, registered under the customers name",local worker,1
1,0 <= ... < 200 DM,48,existing credits paid back duly till now,radio/television,5951,... < 100 DM,1 <= < 4 years,2,female - divorced/separated/married,none,2,real estate,22,none,own,1,skilled employee/official,1,none,local worker,2
2,no checking account,12,critical account/other credits existing (not a...,education,2096,... < 100 DM,4 <= < 7 years,2,male - single,none,3,real estate,49,none,own,1,unskilled - resident,2,none,local worker,1
3,... < 0 DM,42,existing credits paid back duly till now,furniture/equipment,7882,... < 100 DM,4 <= < 7 years,2,male - single,guarantor,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee/official,2,none,local worker,1
4,... < 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= < 4 years,3,male - single,none,4,unknown / no property,53,none,for free,2,skilled employee/official,2,none,local worker,2


In [26]:
# TASK 3:
# создать новую колонку duration_category
# 0-12 short
# 12-24 medium
# 24+ long

# Answer:
from pyspark.sql.functions import col, when

df = df.withColumn("duration_category", when(col("duration") <= 12, "short").when(((col("duration") <= 24) & (col("duration") > 12)), "medium").otherwise("long"))

show(df)

,existingchecking,duration,credithistory,purpose,creditamount,savings,employmentsince,installmentrate,statussex,otherdebtors,residencesince,property,age,otherinstallmentplans,housing,existingcredits,job,peopleliable,telephone,foreignworker,classification,duration_category
0,... < 0 DM,6,critical account/other credits existing (not a...,radio/television,1169,unknown/ no savings account,>= 7 years,4,male - single,none,4,real estate,67,none,own,2,skilled employee/official,1,"yes, registered under the customers name",local worker,1,short
1,0 <= ... < 200 DM,48,existing credits paid back duly till now,radio/television,5951,... < 100 DM,1 <= < 4 years,2,female - divorced/separated/married,none,2,real estate,22,none,own,1,skilled employee/official,1,none,local worker,2,long
2,no checking account,12,critical account/other credits existing (not a...,education,2096,... < 100 DM,4 <= < 7 years,2,male - single,none,3,real estate,49,none,own,1,unskilled - resident,2,none,local worker,1,short
3,... < 0 DM,42,existing credits paid back duly till now,furniture/equipment,7882,... < 100 DM,4 <= < 7 years,2,male - single,guarantor,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee/official,2,none,local worker,1,long
4,... < 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= < 4 years,3,male - single,none,4,unknown / no property,53,none,for free,2,skilled employee/official,2,none,local worker,2,medium


In [33]:
# TASK 4:
# выполнить SQL запрос, который работает по след логике
# categorical_duration = 'short' или categorical_duration = 'medium'

# Answer:
df.createOrReplaceTempView("DF")

output = spark.sql("select * from DF where DF.duration_category='short' or DF.duration_category='medium'")

show(output)

,existingchecking,duration,credithistory,purpose,creditamount,savings,employmentsince,installmentrate,statussex,otherdebtors,residencesince,property,age,otherinstallmentplans,housing,existingcredits,job,peopleliable,telephone,foreignworker,classification,duration_category
0,... < 0 DM,6,critical account/other credits existing (not a...,radio/television,1169,unknown/ no savings account,>= 7 years,4,male - single,none,4,real estate,67,none,own,2,skilled employee/official,1,"yes, registered under the customers name",local worker,1,short
1,no checking account,12,critical account/other credits existing (not a...,education,2096,... < 100 DM,4 <= < 7 years,2,male - single,none,3,real estate,49,none,own,1,unskilled - resident,2,none,local worker,1,short
2,... < 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= < 4 years,3,male - single,none,4,unknown / no property,53,none,for free,2,skilled employee/official,2,none,local worker,2,medium
3,no checking account,24,existing credits paid back duly till now,furniture/equipment,2835,500 <= ... < 1000 DM,>= 7 years,3,male - single,none,4,if not A121 : building society savings agreeme...,53,none,own,1,skilled employee/official,1,none,local worker,1,medium
4,no checking account,12,existing credits paid back duly till now,radio/television,3059,.. >= 1000 DM,4 <= < 7 years,2,male - divorced/separated,none,4,real estate,61,none,own,1,unskilled - resident,1,none,local worker,1,short


# Продолжаем работать с запросами

In [34]:
empl = spark.read \
          .option("delimiter", ",") \
          .load("/content/employee.csv",
                 format='com.databricks.spark.csv', 
                 header='true', 
                 inferSchema='true')

show(empl)

,ssn,fname,lname,sex,salary,super_ssn,dno
0,1,peter,skachovsky,m,1000,2.0,10
1,2,ronald,dehaan,m,2000,4.0,10
2,3,alina,petrova,f,1500,2.0,10
3,4,sergey,paramonov,m,4000,NaN,20
4,5,andra,busoniu,f,2500,NaN,30


In [35]:
works_on = spark.read \
          .option("delimiter", ",") \
          .load("/content/works_on.csv",
                 format='com.databricks.spark.csv', 
                 header='true', 
                 inferSchema='true')
show(works_on)

,essn,pno,hours
0,1,1,10
1,1,2,20
2,1,3,5
3,1,4,10
4,1,5,3


In [36]:
project = spark.read \
          .option("delimiter", ",") \
          .load("/content/project.csv",
                 format='com.databricks.spark.csv', 
                 header='true', 
                 inferSchema='true')
show(project)

,pname,pnumber,plocation,dnum
0,A,1,Dresden,10
1,A,2,Bolzano,10
2,C,3,Moscow,20
3,X,4,Vienna,30
4,Z,5,Houston,20


## Question 1
Question: Retrieve the names of all employees in department 10 who work more than 10 hours per week on the ProductX project!

SQL code to guide:
``` SQL
SELECT Fname, Lname
FROM EMPLOYEE, WORKS_ON, PROJECT 
WHERE Ssn=Essn
AND Pno=Pnumber
AND Dno=10
AND Hours>=10
AND Pname=’X’
```

In [39]:
empl.createOrReplaceTempView("EMPL")
works_on.createOrReplaceTempView("WORKS_ON")
project.createOrReplaceTempView("PROJECT")
output = spark.sql("select Fname, Lname from EMPL, WORKS_ON, PROJECT where ssn=Essn and Pno=Pnumber and Dno=10 and Hours >= 10 and Pname='X'")
show(output)

,Fname,Lname
0,peter,skachovsky


In [42]:
output = empl.join(works_on, empl.ssn == works_on.essn).join(project, project.pnumber == col("pno"))\
.where(col("hours") >=10)\
.where(col("pname")=='X')\
.where(col("Dno")==10)
show(output)

,ssn,fname,lname,sex,salary,super_ssn,dno,essn,pno,hours,pname,pnumber,plocation,dnum
0,1,peter,skachovsky,m,1000,2,10,1,4,10,X,4,Vienna,30


## Question 2
Question: List the names of all employees who have a
dependent with the same first name as themselves

SQL code to guide:
``` SQL
SELECT Fname, Lname
FROM EMPLOYEE, DEPENDENT
WHERE Ssn=Essn
AND Dependent_name=Fname
```

In [43]:
dependent = spark.read \
          .option("delimiter", ",") \
          .load("/content/dependent.csv",
                 format='com.databricks.spark.csv', 
                 header='true', 
                 inferSchema='true')
show(dependent)

,essn,dependent_name,sex
0,1,anna,f
1,2,ronald,m
2,3,evgeny,m


In [47]:
output = dependent.join(empl, dependent.essn == empl.ssn)\
.where(col("dependent_name") == col("Fname")).select(col("fname"), col("lname"))
show(output)

,fname,lname
0,ronald,dehaan


## Question  3
Question: For each project, list the project name and the total hours per week (by all employees) spent on that project

SQL code to guide:
``` SQL
SELECT Pno, Pname, SUM(Hours)
FROM PROJECT JOIN WORKS_ON ON Pno=Pnumber 
GROUP BY Pno, Pname
```

In [61]:
output = project.join(works_on, project.pnumber == works_on.pno).groupby('pno').agg({"hours": "sum"})

In [62]:
show(output)

,pno,sum(hours)
0,1,20
1,3,5
2,5,13
3,4,50
4,2,30


## Question  4

Question: List the last names of all department managers who have no dependents

SQL code to guide:
``` SQL
SELECT Fname, Lname
FROM EMPLOYEE, DEPARTMENT 
WHERE Ssn=Mgr_ssn
AND NOT EXISTS (SELECT *
                FROM DEPENDENT 
                WHERE Essn = Ssn)
```

In [63]:
department = spark.read \
          .option("delimiter", ",") \
          .load("/content/department.csv",
                 format='com.databricks.spark.csv', 
                 header='true', 
                 inferSchema='true')
show(department)

,dname,dnumber,mgr_ssn
0,cs,10,2
1,math,20,4
2,bi,30,5


In [64]:
output = empl.join(department, empl.ssn==department.mgr_ssn).join(dependent, empl.ssn==dependent.essn, "left_anti")
show(output)

,ssn,fname,lname,sex,salary,super_ssn,dno,dname,dnumber,mgr_ssn
0,4,sergey,paramonov,m,4000,NaN,20,math,20,4
1,5,andra,busoniu,f,2500,NaN,30,bi,30,5
